# Vanishing Sphere (Reducing radius over time)
Initialize BoSSS

In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "VanishingSphere2D" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "convergenceAggSteadyStokesExactBC" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient q0-Lichtenberg user storage : mt64qocy@lcluster18.hrz.tu-darmstadt.de, Slurm account: special00006"
1,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient q1-Lichtenberg scratch : mt64qocy@lcluster14.hrz.tu-darmstadt.de, Slurm account: special00006"
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q2-BoSSS-Convergence @DC3, @\\dc3\userspace\toprak\BoSSS-Convergence"
3,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q3-BoSSS-ConvergenceFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConvergenceFinal"
4,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q4-BoSSS-ConditionFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConditionFinal"
5,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q5-BoSSS-ConvSteadyFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConvSteadyFinal"
6,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client LocalPC @C:\localBatch


In [4]:
var myBatch = ExecutionQueues[5];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

Project name is set to 'VanishingSphere2D_convergenceAggSteadyStokesExactBC'.
Opening existing database '\\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC'.


{ Session Count = 30; Grid Count = 10; Path = \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC }

In [5]:
int SpaceDim = 2;
double _partRad = 0.6;
var Gshape = Shape.Sphere;

In [6]:
// set parameterz
int[] core_sweep = { 1};
int[] PolyDegS = new int[] {1, 2, 3};
int[] ResArray = new int[] {15, 30, 60, 120, 240};

In [7]:
int NoOfTimeSteps = 320;
bool Steady = true;
bool IncludeConvection = false;

# Grid

In [8]:
double xMax = 1, yMax = 1, zMax = 1;
double xMin = -1, yMin = -1, zMin = -1;


In [9]:
List<IGridInfo> Grids = new List<IGridInfo>();
if (SpaceDim ==2){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);

    GridCommons grd;
    string gname = "SphereMerger2D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;
    }else{

        grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);

    
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Velocity_Inlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


Looking for SphereMerger2D_1BCs with 225 cells
Found SphereMerger2D_1BCs with 57600 cells on the database
Found SphereMerger2D_1BCs with 14400 cells on the database
Found SphereMerger2D_1BCs with 3600 cells on the database
Found SphereMerger2D_1BCs with 900 cells on the database
Found SphereMerger2D_1BCs with 225 cells on the database
Found SphereMerger2D_1BCs with 57600 cells on the database
Found SphereMerger2D_1BCs with 14400 cells on the database
Found SphereMerger2D_1BCs with 3600 cells on the database
Found SphereMerger2D_1BCs with 900 cells on the database
Found SphereMerger2D_1BCs with 225 cells on the database
Found 2 times grids with the same configuration for the Res=15
Chosing the first one found: SphereMerger2D_1BCs
Looking for SphereMerger2D_1BCs with 900 cells
Found SphereMerger2D_1BCs with 57600 cells on the database
Found SphereMerger2D_1BCs with 14400 cells on the database
Found SphereMerger2D_1BCs with 3600 cells on the database
Found SphereMerger2D_1BCs with 900 cel

#0: { Guid = 8c288de8-9725-42da-b8af-5e92905f7e99; Name = SphereMerger2D_1BCs; Cell Count = 225; Dim = 2 }
#1: { Guid = 0602948d-d3f4-4ae8-88a8-d380fb4b3420; Name = SphereMerger2D_1BCs; Cell Count = 900; Dim = 2 }
#2: { Guid = a65b8da8-6d2e-4e17-8d6f-5d2c1476a17f; Name = SphereMerger2D_1BCs; Cell Count = 3600; Dim = 2 }
#3: { Guid = 198b57b3-9137-4e60-a96a-4110ed3baf49; Name = SphereMerger2D_1BCs; Cell Count = 14400; Dim = 2 }
#4: { Guid = 522a1fba-e0c5-4c12-9f7b-79f8942e5aab; Name = SphereMerger2D_1BCs; Cell Count = 57600; Dim = 2 }


In [10]:
if (SpaceDim ==3){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "SphereMerger_3D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;

    }else{

        grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes,_zNodes);   
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Velocity_Inlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


#0: { Guid = 8c288de8-9725-42da-b8af-5e92905f7e99; Name = SphereMerger2D_1BCs; Cell Count = 225; Dim = 2 }
#1: { Guid = 0602948d-d3f4-4ae8-88a8-d380fb4b3420; Name = SphereMerger2D_1BCs; Cell Count = 900; Dim = 2 }
#2: { Guid = a65b8da8-6d2e-4e17-8d6f-5d2c1476a17f; Name = SphereMerger2D_1BCs; Cell Count = 3600; Dim = 2 }
#3: { Guid = 198b57b3-9137-4e60-a96a-4110ed3baf49; Name = SphereMerger2D_1BCs; Cell Count = 14400; Dim = 2 }
#4: { Guid = 522a1fba-e0c5-4c12-9f7b-79f8942e5aab; Name = SphereMerger2D_1BCs; Cell Count = 57600; Dim = 2 }


## Generate Control object

### governing equations
- incompressible transient Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t) = \vec{u}_{body} = \boldsymbol{\omega}(t) \times \vec{r} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$
<br>$p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 



- angular velocity of rotating sphere $\boldsymbol{\omega}(t)=\frac{Re*\mu_A}{\rho_A*r_{p}*(2*r_p)}$

In [11]:
Func<IGridInfo, int,double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = 
delegate(IGridInfo grd, int k, double viscosity, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;

    //Session Settings
    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    C.ImmediatePlotPeriod = 1;
    C.SuperSampling = 0;    

    // Physical Parameters
    // =================== 
    const double rhoA = 1;
    const double Re = 500;
    double muA = viscosity;
    double partRad = _partRad;
    double partDia = 2*partRad;
    double VelocityMax = Re * muA / rhoA / partDia;
    double anglev = VelocityMax / partRad;
    double period = 2*Math.PI/anglev;
    double ts = period/NoOfTimeSteps;
    double[] pos = new double[SpaceDim];
    string rotAxis= RotAxis ?? "z";
    
    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    // Meta data
    // ============    
    C.SessionName = string.Format("J{0}_k{1}_agg{2}_AMR{3}_LB{4}_at{5}_r{6}_W{7:f2}_{8}D",
                     J, k,AggThrs,AMR,LoadBalance,rotAxis.ToUpper(),partRad,anglev,SpaceDim);

    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees and configuration
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 8)); // (velocity degree,level set) (pressure degree = k-1)
    C.saveperiod = 10;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = false;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;

    // rigid body and boundary conditions
    // ===================
    double rateOfRadius = Steady ? 0 : -1/(double)NoOfTimeSteps/ts;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim, rateOfRadius: rateOfRadius, staticShape: Steady);
    C.SessionName += $"dRdt{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis(rotAxis);
    C.UseImmersedBoundary = true;
    C.Rigidbody.AssignExactSolutionForSteadyRotatingSphere();
    // Set initial and boundary conditions
    // ============
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    
    var codeVelX = $"(X, t) => {-anglev*partRad*partRad}*X[1]/(X[0]*X[0]+X[1]*X[1])";
    var codeVelY = $"(X, t) => {anglev*partRad*partRad}*X[0]/(X[0]*X[0]+X[1]*X[1])";

    C.AddBoundaryValue("Velocity_Inlet", "VelocityX", new Formula(codeVelX,true));
    C.AddBoundaryValue("Velocity_Inlet", "VelocityY", new Formula(codeVelY,true));

    // solver options
    // ===================
    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None; // idle level set (used for evolving LS)
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = LinearSolverCode.direct_mumps.GetConfig(); //exp_Kcycle_schwarz.GetConfig();
    C.SessionName += "_MUMPS";
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 0;
    C.NonLinearSolver.MaxSolverIterations = 30;
    C.NonLinearSolver.verbose = true;
    C.AdaptiveMeshRefinement = AMR;

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = Double.MaxValue;
        C.NoOfTimesteps = 1;
        C.SessionName += "_Steady";
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps; 
    }
    
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler; 
    C.dtFixed = dt;
    C.SkipSolveAndEvaluateResidual = false; //to save computational time for condition study
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;
    C.PlotAgglomeration = false;

    // //Condition number study
    // // ============
    // var config = new OperatorAnalysisConfig();
    // config.CalculateMassMatrix = true;
    // config.CalculateGlobalConditionNumbers = true;
    // config.CalculateStencilConditionNumbers = false;
    // config.PlotStencilConditionNumbers = false;
    // C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
    // C.SessionName += "_condStudy";


    // L2 error with exact solution
    // ============
    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.L2ErrorLogger());
    //C.SessionName += "_re";

    // reference values (hardcoded)
    // C.ExactSolutionVelocity = new Dictionary<string, Func<double[], double, double>[]>();
    // C.ExactSolutionPressure = new Dictionary<string, Func<double[], double, double>>();    

    // var code = $"(X, t) => {-anglev*partRad*partRad}*X[1]/(X[0]*X[0]+X[1]*X[1]), (X, t) => {anglev*partRad*partRad}*X[0]/(X[0]*X[0]+X[1]*X[1])";
    
    // var my_formula =  new Formula(code);
    // C.ExactSolutionVelocity.Add("A", new Func<double[], double, double>[] { (X, t) => -anglev*partRad*partRad*X[1]/(X[0]*X[0]+X[1]*X[1]), (X, t) => anglev*partRad*partRad*X[0]/(X[0]*X[0]+X[1]*X[1]) });

    // C.ExactSolutionVelocity.Add("C", new Func<double[], double, double>[] { (X, t) => 0, (X, t) => 0 });
    return C;    
};

In [12]:
//GenXNSECtrl(Grids[0],1,1,"z",false,false,0)

# Submit jobs into cluster
Create the control objects for the sessions

In [13]:
var ViscositySweep = new double[]{1E-2};
var axes = new string[]{"z"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};
var aggThresolds = new double[] {0.2, 0.3, 0.4, 0.5};

List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids)
    foreach(int k in PolyDegS)
            foreach(double v in ViscositySweep)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
            {
                controls.Add(GenXNSECtrl(grd,k,v,axis,AMR,LoadBalance,agg));
            }


List sessions to be submitted

In [14]:
Console.WriteLine($"Total Number of Control Files: {controls.Count}");
controls.Summary()

Total Number of Control Files: 60


List of 60 instances of XNSE_Control {
  0: BoSSS.Application.XNSE_Solver.XNSE_Control
  1: BoSSS.Application.XNSE_Solver.XNSE_Control
  2: BoSSS.Application.XNSE_Solver.XNSE_Control
  3: BoSSS.Application.XNSE_Solver.XNSE_Control
  4: BoSSS.Application.XNSE_Solver.XNSE_Control
  5: BoSSS.Application.XNSE_Solver.XNSE_Control
  6: BoSSS.Application.XNSE_Solver.XNSE_Control
  7: BoSSS.Application.XNSE_Solver.XNSE_Control
  8: BoSSS.Application.XNSE_Solver.XNSE_Control
  9: BoSSS.Application.XNSE_Solver.XNSE_Control
  10: BoSSS.Application.XNSE_Solver.XNSE_Control
  11: BoSSS.Application.XNSE_Solver.XNSE_Control
  12: BoSSS.Application.XNSE_Solver.XNSE_Control
  13: BoSSS.Application.XNSE_Solver.XNSE_Control
  14: BoSSS.Application.XNSE_Solver.XNSE_Control
  15: BoSSS.Application.XNSE_Solver.XNSE_Control
  16: BoSSS.Application.XNSE_Solver.XNSE_Control
  17: BoSSS.Application.XNSE_Solver.XNSE_Control
  18: BoSSS.Application.XNSE_Solver.XNSE_Control
  19: BoSSS.Application.XNSE_Solver.XNSE

Submit jobs

In [15]:
int k = 0;
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores;            
            var aJob   = new Job("Vanishing"+Gshape+"_"+ctrl.SessionName,typeof(XNSE));
            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            //aJob.ExecutionTime            = "72:00:00";
            aJob.UseComputeNodesExclusive = true;
            aJob.NumberOfThreads = 4;
            aJob.Activate(myBatch);
            
            Console.WriteLine("Submitted {0}th \n",k) ;
            k++;           
        }       
    }


Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphere_SolverTrue_J225_k1_agg0.2_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC-XNSE_Solver2024Aug20_124835.242803
copied 43 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.
Submitted 0th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphere_SolverTrue_J225_k1_agg0.3_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC-XNSE_Solver2024Aug20_124849.853049
copied 43 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.
Submitted 1th 

Deployments so far (0): ;
Success: 0
job submit count: 0


...

Deploying job VanishingSphere_SolverTrue_J57600_k3_agg0.3_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC-XNSE_Solver2024Aug20_130222.887557
copied 43 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.
Submitted 57th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphere_SolverTrue_J57600_k3_agg0.4_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC-XNSE_Solver2024Aug20_130239.958073
copied 43 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.
Submitted 58th 

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job VanishingSphere_SolverTrue_J57600_k3_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC-XNSE_Solver2024Aug20_130254.697490
copied 43 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.
Submitted 59th 



List sessions submitted

In [16]:
myDB.Sessions


#0: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.4_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1*	8/20/2024 1:02:57 PM	08a39405...
#1: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1*	8/20/2024 1:02:52 PM	fa95be63...
#2: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.2_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1*	8/20/2024 1:03:04 PM	4e6c9e76...
#3: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.3_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1*	8/20/2024 1:02:20 PM	283173f3...
#4: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.2_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1*	8/20/2024 1:02:28 PM	4a8f2666...
#5: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J14400_k

In [17]:
//s.Export().Do();